In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

# Load the DataSet

In [2]:
(X_train,y_train), (X_test,y_test) = tf.keras.datasets.cifar10.load_data()

170498071/170498071 [==============================] - 6s 0us/step


In [3]:
tf.config.experimental.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
X_train.shape

(50000, 32, 32, 3)

In [5]:
!ls

sample_data


In [7]:
%cd sample_data

/content/sample_data


In [9]:
!dir

anscombe.json		      mnist_test.csv
california_housing_test.csv   mnist_train_small.csv
california_housing_train.csv  README.md


In [10]:
classes = ["airplane","automobile","bird","cat","deer","dog","frog","horse","ship","truck"]

In [11]:
X_train_scaled = X_train / 255
X_test_scaled = X_test / 255

In [12]:
y_train_categorical = keras.utils.to_categorical(
    y_train, num_classes=10, dtype='float32'
)
y_test_categorical = keras.utils.to_categorical(
    y_test, num_classes=10, dtype='float32'
)

In [13]:
def get_model():
    model = keras.Sequential([
            keras.layers.Flatten(input_shape=(32,32,3)),
            keras.layers.Dense(3000, activation='relu'),
            keras.layers.Dense(1000, activation='relu'),
            keras.layers.Dense(10, activation='sigmoid')    
        ])

    model.compile(optimizer='SGD',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [14]:
train_tf_dataset = tf.data.Dataset.from_tensor_slices((X_train_scaled, y_train_categorical))
test_tf_dataset = tf.data.Dataset.from_tensor_slices((X_test_scaled, y_test_categorical))

In [15]:
strategy = tf.distribute.MirroredStrategy()

In [16]:
strategy.num_replicas_in_sync

1

In [17]:
BATCH_SIZE_PER_REPLICA = 1000
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync


train_dataset = train_tf_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_tf_dataset.batch(BATCH_SIZE)

In [18]:
%%timeit -n1 -r1 
with strategy.scope():
    gpu_model = get_model()
    gpu_model.fit(train_dataset, epochs=50)

Epoch 1/50
50/50 [==============================] - 4s 20ms/step - loss: 2.1370 - accuracy: 0.2461
Epoch 2/50
50/50 [==============================] - 1s 20ms/step - loss: 1.9720 - accuracy: 0.3192
Epoch 3/50
50/50 [==============================] - 1s 21ms/step - loss: 1.9010 - accuracy: 0.3444
Epoch 4/50
50/50 [==============================] - 1s 19ms/step - loss: 1.8575 - accuracy: 0.3601
Epoch 5/50
50/50 [==============================] - 1s 19ms/step - loss: 1.8267 - accuracy: 0.3709
Epoch 6/50
50/50 [==============================] - 1s 20ms/step - loss: 1.8030 - accuracy: 0.3794
Epoch 7/50
50/50 [==============================] - 1s 20ms/step - loss: 1.7835 - accuracy: 0.3859
Epoch 8/50
50/50 [==============================] - 1s 21ms/step - loss: 1.7669 - accuracy: 0.3916
Epoch 9/50
50/50 [==============================] - 1s 21ms/step - loss: 1.7522 - accuracy: 0.3968
Epoch 10/50
50/50 [==============================] - 1s 19ms/step - loss: 1.7389 - accuracy: 0.4008
Epoch 11/